<h1 align="center">Практичиское задание 2: Sparse Variational Dropout</h1>

- В этом задании вам предстоит реализовать Sparse VD -- метод для разреживания нейронных сетей https://arxiv.org/abs/1701.05369  
- Для того чтобы сдать задание нужно прислать заполненный и рабочий ipython-notebook в anytask и туда-же вопросы можно задавать
- **Добавьте фамилию в название файла:** SparseVD-assignment-vetrov.ipynb
- **Напишите контактные данные, чтобы можно было с тобой связаться:** Имя: 123, Фамилия: 123, Почта: 123

In [ ]:
import math
import torch
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from logger import Logger
from torch.nn import Parameter
from torch.autograd import Variable
from torchvision import datasets, transforms

## Реализуем полносвязный Sparse VD слой

In [ ]:
class LinearSVDO(nn.Module):
    def __init__(self, in_features, out_features, threshold, bias=True):
        super(LinearSVDO, self).__init__()
        # in_features int
        # out_features int 
        # threshold float
        self.in_features = in_features
        self.out_features = out_features
        self.threshold = threshold

        # =======================================
        # Создайте параметры модели -- объекты класса Parameter
        # W размера (out_features x in_features)
        # log_sigma размера (out_features x in_features)
        # bias размера (1, out_features)
        # =======================================
        
        self.reset_parameters()

    def reset_parameters(self):
        # =======================================
        # Инициализируйте параметры модели
        # W -- нормальный случайный шум с центром в 0 и маленькой дисперсией
        # log_sigma -- маленьким значением ~ -5 
        # bias -- можно 0
        # =======================================    
        
    def forward(self, x):
        # =======================================
        # x: Variable containing: [torch.FloatTensor of size batch_size x in_features]
        # Return: type: Variable containing [torch.FloatTensor of size batch_size x out_features]
        # ----------------------------------------
        # Тут нужно написать forward шаг для Sparse VD слоя для минибатча объектов x
        # На этапе обучения: Вернуть семпл активаций с помощью Local Reparametrization Trick 
        # На этапе тестирования: Вернуть средние активации, посчитанные с обрезанными весами 
        # Правило обрезания весов: alpha_ij < self.threshold ====> w_ij = 0
        # ----------------------------------------
        # Клипинг alpha_ij, например torch.clamp(self.log_alpha, -10, 10) может улучшить стабильность 
        # Чтобы не встретить nan-ы используйте трюк log(a) = log(a + 1e-8)
        # ======================================= 
        
    def kl_reg(self):
        # =======================================
        # Вернуть суммарную KL дивергенцию для всех параметров слоя 
        # Return: Variable containing: [torch.FloatTensor of size 1]
        # =======================================

## Создадим простую архитектуру LeNet-300-100

In [ ]:
class Net(nn.Module):
    def __init__(self, threshold):
        super(Net, self).__init__()
        self.fc1 = LinearSVDO(28*28, 300, threshold)
        self.fc2 = LinearSVDO(300,  100, threshold)
        self.fc3 = LinearSVDO(100,  10, threshold)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

## Загрузим MNIST

In [ ]:
def get_mnist(batch_size):
    trsnform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
        transform=trsnform), batch_size=batch_size, shuffle=True)

    return train_loader, test_loader

## Определим новую функцию потерь SGVLB

In [ ]:
class SGVLB(nn.Module):
    def __init__(self, net, train_size):
        super(SGVLB, self).__init__()
        self.train_size = train_size
        self.net = net

    def forward(self, output, target, kl_weight=1.0):
        # =======================================
        # output -- ответы модели для минибатча [torch.FloatTensor of size batch_size x 10]
        # target -- настоящие ответы  [torch.LongTensor of size batch_size]
        # kl_weight -- коэффициент на который нужно умножить kl дивергенцию, нужен для отжига (читай ниже)
        # Вернуть Variable с посчитанной SGVLB функцией потерь 
        # Используйте self.net.children() для обхода всех слоев модели
        # !!! Проверьте что множитель перед data term правильный !!!
        # Return: Variable containing: [torch.FloatTensor of size 1]
        # =======================================

In [ ]:
epochs, batch_size, threshold = 100, 100, 3
model = Net(threshold)
optimizer = # Тут ваш любимый оптимизатор, адам -- хороший выбор
scheduler = # Тут расписание шага обучения torch.optim.lr_scheduler
fmt = {'tr_los': '3.1e', 'te_loss': '3.1e', 'sp_0': '.3f', 'sp_1': '.3f', 'lr': '3.1e', 'kl': '.2f'}
logger = Logger('sparse_vd', fmt=fmt)

In [ ]:
# Для ускорения обучения используйте отжиг kl 
# Отжиг кл помогает избежать "плохих локальных оптимумов" и устроен так 
# kl_weight стартует с маленького значения и увеличивается до 1 с каким-то шагом
# а потом остается 1 до конца обучения 

train_loader, test_loader = get_mnist(batch_size)
sgvlb = SGVLB(model, len(train_loader.dataset))
step = # шаг для увеличения kl_weight

# ===============
kl_weight = step # Начальное значение веса KL
# ===============
for epoch in range(1, epochs + 1):
    scheduler.step()
    model.train()
    train_loss, train_acc = 0, 0 
    kl_weight = min(kl_weight+step, 1) 
    logger.add_scalar(epoch, 'kl', kl_weight)
    logger.add_scalar(epoch, 'lr', scheduler.get_lr()[0])
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).view(-1, 28*28), Variable(target)
        optimizer.zero_grad()
        
        output = model(data)
        pred = output.data.max(1)[1] 
        loss = sgvlb(output, target, kl_weight)
        loss.backward()
        optimizer.step()
        
        train_loss += loss 
        train_acc += np.sum(pred.numpy() == target.data.numpy())

    logger.add_scalar(epoch, 'tr_los', train_loss / len(train_loader.dataset))
    logger.add_scalar(epoch, 'tr_acc', train_acc / len(train_loader.dataset) * 100)
    
    
    model.eval()
    test_loss, test_acc = 0, 0
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = Variable(data, volatile=True).view(-1, 28*28), Variable(target)
        output = model(data)
        test_loss += sgvlb(output, target, kl_weight).data[0] 
        pred = output.data.max(1)[1] 
        test_acc += np.sum(pred.numpy() == target.data.numpy())
        
    logger.add_scalar(epoch, 'te_loss', test_loss / len(test_loader.dataset))
    logger.add_scalar(epoch, 'te_acc', test_acc / len(test_loader.dataset) * 100)
    
    for i, c in enumerate(model.children()):
        if hasattr(c, 'kl_reg'):
            logger.add_scalar(epoch, 'sp_%s' % i, (c.log_alpha.data.numpy() > threshold).mean())
            
    logger.iter_info()

In [ ]:
# Посмотрим во сколько раз у нас уменьшилось количество весов в первом слое 
# Тут хотим получить в 30+ раз меньше весов без падения качества на тесте

all_w, kep_w = 0, 0

for c in model.children():
    kep_w += (c.log_alpha.data.numpy() < 3).sum()
    all_w += c.log_alpha.data.numpy().size

print('keept weight ratio =', all_w/kep_w)

In [ ]:
# Посмотрим какая компрессия получилась на диске
import scipy
import numpy as np
from scipy.sparse import csc_matrix, csc_matrix, coo_matrix, dok_matrix

row, col, data = [], [], []
M = list(model.children())[0].W.data.numpy()
LA = list(model.children())[0].log_alpha.data.numpy()

for i in range(300):
    for j in range(28*28):
        if LA[i, j] < 3:
            row += [i]
            col += [j]
            data += [M[i, j]]

Mcsr = csc_matrix((data, (row, col)), shape=(300, 28*28))
Mcsc = csc_matrix((data, (row, col)), shape=(300, 28*28))
Mcoo = coo_matrix((data, (row, col)), shape=(300, 28*28))

In [ ]:
np.savez_compressed('M_w', M)
scipy.sparse.save_npz('Mcsr_w', Mcsr)
scipy.sparse.save_npz('Mcsc_w', Mcsc)
scipy.sparse.save_npz('Mcoo_w', Mcoo)

In [ ]:
ls -lah | grep _w

## Бонусная часть баллов 25%

- Сжать этим методом сверточную сеть LeNet-5-Caffe в 100 + раз http://caffe.berkeleyvision.org/gathered/examples/mnist.html
- Поэкспериментировать с разной битностью весов -- при какой битности качество начинает падать
- Помогают ли веса меньшей битности сэкономить место на диске?